In [1]:
import numpy as np

In [3]:
import numpy as np

class NN:
    def __init__(self, input_dim=2, hidden_layers=[3], output_dim=1):

        self.weights = []
        self.bias = []

       
        self.weights.append(
            np.random.normal(
                0,
                scale=np.sqrt(6 / (input_dim + hidden_layers[0])),
                size=(input_dim, hidden_layers[0])
            )
        )

        
        for i in range(len(hidden_layers) - 1):
            self.weights.append(
                np.random.normal(
                    0,
                    scale=np.sqrt(6 / (hidden_layers[i] + hidden_layers[i+1])),
                    size=(hidden_layers[i], hidden_layers[i+1])
                )
            )

       
        self.weights.append(
            np.random.normal(
                0,
                scale=np.sqrt(6 / (hidden_layers[-1] + output_dim)),
                size=(hidden_layers[-1], output_dim)
            )
        )

        for i in range(len(hidden_layers)):
            self.bias.append(np.random.randn(hidden_layers[i]))

        
        self.bias.append(np.random.randn(output_dim))


    def activation(self,Z):
        return 1/(1+np.exp(-Z))   # sigmoid
    
    def der_activation(self,A):
        return A*(1-A)            # derivative wrt activated value

    def forwardProp(self,X):
        self.actvs=[X]

        for i in range(len(self.weights)):
            z= self.actvs[-1] @ self.weights[i]+self.bias[i]
            a= self.activation(z)
            self.actvs.append(a)

        return self.actvs[-1]

    
    def backProp(self,X,y,lr=0.01,epochs=100):
        for e in range(epochs):
            y_pred = self.forwardProp(X)

            deltas=[(y_pred-y) * self.der_activation(self.actvs[-1])]
            grads=[self.actvs[-2].T @ deltas[-1]]

            for i in range(len(self.weights)-2,-1,-1):
                d = (deltas[-1] @ self.weights[i+1].T) * self.der_activation(self.actvs[i+1])
                deltas.append(d)
                grads.append(self.actvs[i].T @ d)

            wn = len(self.weights)-1
            for i in range(len(self.weights)):
                self.weights[i] -= lr * grads[wn]
                self.bias[i]    -= lr * np.sum(deltas[wn],axis=0)
                wn -= 1

            if e%10==0:
                loss = np.mean((y_pred-y)**2)
                print(f"Epoch {e}, Loss: {loss:.4f}")


In [4]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
X,y=make_classification(n_samples=2000,
                        n_features=10,
                        n_informative=6,
                        n_redundant=2,
                        n_classes=2,
                        random_state=42)

In [6]:
y=y.reshape(-1,1)

In [7]:
y

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
nn2=NN(input_dim=X_train.shape[1],hidden_layers=[8,6],output_dim=1)
nn2.backProp(X_train,y_train,lr=0.0125,epochs=5000)

Epoch 0, Loss: 0.3355
Epoch 10, Loss: 0.2858
Epoch 20, Loss: 0.2235
Epoch 30, Loss: 0.1073
Epoch 40, Loss: 0.1288
Epoch 50, Loss: 0.1027
Epoch 60, Loss: 0.0862
Epoch 70, Loss: 0.0867
Epoch 80, Loss: 0.0744
Epoch 90, Loss: 0.0649
Epoch 100, Loss: 0.0580
Epoch 110, Loss: 0.0763
Epoch 120, Loss: 0.0535
Epoch 130, Loss: 0.0514
Epoch 140, Loss: 0.0520
Epoch 150, Loss: 0.0794
Epoch 160, Loss: 0.0490
Epoch 170, Loss: 0.0477
Epoch 180, Loss: 0.0465
Epoch 190, Loss: 0.0624
Epoch 200, Loss: 0.0473
Epoch 210, Loss: 0.0446
Epoch 220, Loss: 0.0433
Epoch 230, Loss: 0.0696
Epoch 240, Loss: 0.0426
Epoch 250, Loss: 0.0546
Epoch 260, Loss: 0.0454
Epoch 270, Loss: 0.0450
Epoch 280, Loss: 0.0469
Epoch 290, Loss: 0.0438
Epoch 300, Loss: 0.0442
Epoch 310, Loss: 0.0430
Epoch 320, Loss: 0.0423
Epoch 330, Loss: 0.0418
Epoch 340, Loss: 0.0409
Epoch 350, Loss: 0.0405
Epoch 360, Loss: 0.0398
Epoch 370, Loss: 0.0394
Epoch 380, Loss: 0.0388
Epoch 390, Loss: 0.0386
Epoch 400, Loss: 0.0381
Epoch 410, Loss: 0.0377
Epo

In [43]:
y_pred_test=nn2.forwardProp(X_test)
y_pred_labels=(y_pred_test>0.5).astype(int)
#print(accuracy_score(y_test,y_pred_labels))
#print(y_pred_labels)
print(y_pred_test)

[[9.97393957e-01]
 [9.97208686e-01]
 [9.99338204e-01]
 [3.44568255e-03]
 [2.80872474e-02]
 [4.71064067e-05]
 [3.32069381e-03]
 [9.97570105e-01]
 [1.78061938e-02]
 [1.29873126e-02]
 [2.06664768e-08]
 [3.18373639e-03]
 [9.97438079e-01]
 [9.99335490e-01]
 [9.98586723e-01]
 [3.91955585e-04]
 [3.32070237e-03]
 [1.28318920e-03]
 [5.33746267e-05]
 [9.97394445e-01]
 [4.68003155e-09]
 [1.63672062e-03]
 [9.99977701e-01]
 [9.97429046e-01]
 [9.97649472e-01]
 [1.76973512e-02]
 [9.97670500e-01]
 [1.28162855e-03]
 [9.97492277e-01]
 [9.99337782e-01]
 [1.39411942e-04]
 [9.97407782e-01]
 [9.97394445e-01]
 [9.97931568e-01]
 [9.97572998e-01]
 [9.97394444e-01]
 [9.99338235e-01]
 [9.99312604e-01]
 [9.99332649e-01]
 [9.97394420e-01]
 [5.02520693e-02]
 [9.97730398e-01]
 [9.97438077e-01]
 [9.99139401e-01]
 [9.97394447e-01]
 [1.31795782e-02]
 [1.26123659e-03]
 [9.97394014e-01]
 [9.97394449e-01]
 [1.03320113e-02]
 [9.97961887e-01]
 [9.97713069e-01]
 [9.97643956e-01]
 [9.97590996e-01]
 [5.64263552e-06]
 [9.974954

In [40]:
y_pred_test

array([[9.97393957e-01],
       [9.97208686e-01],
       [9.99338204e-01],
       [3.44568255e-03],
       [2.80872474e-02],
       [4.71064067e-05],
       [3.32069381e-03],
       [9.97570105e-01],
       [1.78061938e-02],
       [1.29873126e-02],
       [2.06664768e-08],
       [3.18373639e-03],
       [9.97438079e-01],
       [9.99335490e-01],
       [9.98586723e-01],
       [3.91955585e-04],
       [3.32070237e-03],
       [1.28318920e-03],
       [5.33746267e-05],
       [9.97394445e-01],
       [4.68003155e-09],
       [1.63672062e-03],
       [9.99977701e-01],
       [9.97429046e-01],
       [9.97649472e-01],
       [1.76973512e-02],
       [9.97670500e-01],
       [1.28162855e-03],
       [9.97492277e-01],
       [9.99337782e-01],
       [1.39411942e-04],
       [9.97407782e-01],
       [9.97394445e-01],
       [9.97931568e-01],
       [9.97572998e-01],
       [9.97394444e-01],
       [9.99338235e-01],
       [9.99312604e-01],
       [9.99332649e-01],
       [9.97394420e-01],


In [24]:
nn.forwardProp(X)

array([[0.90961062],
       [0.91531117],
       [0.91532751],
       [0.91517724]])